# 주식 관련정보 크롤링

### 목차
- 1. 국내
- 2. 해외
    - 2-1. Marketscreener
    - 2-2. Yahoo Finance
- WICS 통합본 내보내기
- 3. 기타 그외
    - 3-1. KOSPI

----------

# 1. 국내 (WICS_KR)

### 라이브러리 가져오기

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 데이터 가져오기

In [ ]:
iem_info = pd.read_csv('./Raw_Data/2_iem_info.csv')
iem_info['iem_cd'] = iem_info['iem_cd'].str.rstrip()

### 크롤링

In [ ]:
def get_wics(lines):
    for line in lines:
        if 'WICS' in line.text:
            return line.text.split('WICS : ')[1]

In [ ]:
def get_sector_wics(company_code):
    url = 'https://navercomp.wisereport.co.kr/v2/company/c1090001.aspx?cmp_cd=' + company_code + '&cn='
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    lines = soup.find_all('dt', {'class':'line-left'})
    wics = get_wics(lines)
    return wics

In [ ]:
x = []
for i in range(len(iem_info)):
    if iem_info.iloc[i, 0].find('A')==0:
        x.append(iem_info.iloc[i, 0][1:7])
for i in range(len(x)):
    if x[i].find('U')==0:
        x[i]=0
while 0 in x:
    x.remove(0)

In [ ]:
wics_kr = pd.DataFrame(None, columns=['iem_cd', 'WICS'], index=range(len(x)))
for i in range(len(x)):
    w = get_sector_wics(x[i])
    wics_kr.iloc[i, 0] = 'A' + x[i]
    wics_kr.iloc[i, 1] = w

### 분류하기 (대분류:cat_1, 중분류: cat_2, 소분류: cat_3)

In [ ]:
wics_kr['cat_1'] = None
wics_kr['cat_2'] = None
wics_kr.columns = ['iem_cd', 'cat_3', 'cat_1', 'cat_2']
wics_kr = wics_kr[['iem_cd', 'cat_1', 'cat_2', 'cat_3']]

wics_kr = pd.merge(wics_kr, iem_info, how='left', on='iem_cd')

In [ ]:
#국내 ETF 분류하기
for i in range(len(wics_kr)):
    if ('ETF' in wics_kr.iloc[i, 4]) | ('지수' in wics_kr.iloc[i, 5]):
        wics_kr.iloc[i, 1] = 'ETF'
        wics_kr.iloc[i, 2] = 'ETF(일반)'
        wics_kr.iloc[i, 3] = 'ETF(1배)'

for i in range(len(wics_kr)):
    if ('INVERSE' in wics_kr.iloc[i, 4]) | ('Inverse' in wics_kr.iloc[i, 4]) | ('inverse' in wics_kr.iloc[i, 4]):
        wics_kr.iloc[i, 1] = 'ETF'
        wics_kr.iloc[i, 2] = 'ETF(인버스)'
        wics_kr.iloc[i, 3] = 'ETF(1배)'
    elif ('LEVERAGE' in wics_kr.iloc[i, 4]) | ('Leverage' in wics_kr.iloc[i, 4]) | ('leverage' in wics_kr.iloc[i, 4]):
        wics_kr.iloc[i, 1] = 'ETF'
        wics_kr.iloc[i, 2] = 'ETF(일반)'
        wics_kr.iloc[i, 3] = 'ETF(2배)'

for i in range(len(wics_kr)):
    if ('2X' in wics_kr.iloc[i, 4]):
        wics_kr.iloc[i, 3] = 'ETF(2배)'

for i in range(len(wics_kr)):
    if '인수목적' in wics_kr.iloc[i, 5]:
        wics_kr.iloc[i, 3] = '기타금융'

In [ ]:
#소분류 세부수정
wics_kr.iloc[[30, 42, 268, 435, 481, 555, 597, 609, 660, 729, 818, 880, 1062, 1072, 1136, 1173, 1300, 1512, 1569, 1580,
           1753, 1810, 1855, 1858, 1865, 1947, 1973, 2007, 2037, 2038, 2073, 2122, 2309, 2474, 2485], 3
         ] = ['건설', '건설', '전자장비와기기', '은행', '건축자재', '호텔,레스토랑,레저', '전자장비와기기', '화학', '반도체와반도체장비',
              '식품', '섬유,의류,신발,호화품', '건축자재', '반도체와반도체장비', '해운사', '자동차부품', '반도체와반도체장비', '종이와목재',
              '생명보험', '에너지장비및서비스', '비철금속', '식품', '건설', 'IT서비스', '백화점과일반상점', '섬유,의류,신발,호화품',
              '섬유,의류,신발,호화품', '식품', '비철금속', '디스플레이장비및부품', '디스플레이장비및부품', '해운사', '가정용기기와용품',
              '섬유,의류,신발,호화품', '비철금속', '인터넷과카탈로그소매']

In [ ]:
#중분류
for i in range(len(wics_kr)):
    if wics_kr.iloc[i, 3] in ['에너지장비및서비스', '석유와가스']:
        wics_kr.iloc[i, 2] = '에너지'
    elif wics_kr.iloc[i, 3] in ['화학', '포장재', '비철금속', '철강', '종이와목재']:
        wics_kr.iloc[i, 2] = '소재'
    elif wics_kr.iloc[i, 3] in ['우주항공과국방', '건축제품', '건축자재', '건설', '가구', '전기장비', '복합기업', '기계', '조선', '무역회사와판매업체']:
        wics_kr.iloc[i, 2] = '자본재'
    elif wics_kr.iloc[i, 3] in ['상업서비스와공급품']:
        wics_kr.iloc[i, 2] = '상업서비스와공급품'
    elif wics_kr.iloc[i, 3] in ['항공화물운송과물류', '항공사', '해운사', '도로와철도운송', '운송인프라']:
        wics_kr.iloc[i, 2] = '운송'
    elif wics_kr.iloc[i, 3] in ['자동차부품', '자동차']:
        wics_kr.iloc[i, 2] = '자동차와부품'
    elif wics_kr.iloc[i, 3] in ['가정용기기와용품', '레저용장비와제품', '섬유,의류,신발,호화품', '화장품', '문구류']:
        wics_kr.iloc[i, 2] = '내구소비재와의류'
    elif wics_kr.iloc[i, 3] in ['호텔,레스토랑,레저', '다각화된소비자서비스']:
        wics_kr.iloc[i, 2] = '호텔,레스토랑,레저등'
    elif wics_kr.iloc[i, 3] in ['판매업체', '인터넷과카탈로그소매', '백화점과일반상점', '전문소매']:
        wics_kr.iloc[i, 2] = '소매(유통)'
    elif wics_kr.iloc[i, 3] in ['교육서비스']:
        wics_kr.iloc[i, 2] = '교육서비스'
    elif wics_kr.iloc[i, 3] in ['식품과기본식료품소매']:
        wics_kr.iloc[i, 2] = '식품과기본식료품소매'
    elif wics_kr.iloc[i, 3] in ['음료', '식품', '담배']:
        wics_kr.iloc[i, 2] = '식품,음료,담배'
    elif wics_kr.iloc[i, 3] in ['가정용품']:
        wics_kr.iloc[i, 2] = '가정용품과개인용품'
    elif wics_kr.iloc[i, 3] in ['건강관리장비와용품', '건강관리업체및서비스', '건강관리기술']:
        wics_kr.iloc[i, 2] = '건강관리장비와서비스'
    elif wics_kr.iloc[i, 3] in ['생물공학', '제약', '생명과학도구및서비스']:
        wics_kr.iloc[i, 2] = '제약과생물공학'
    elif wics_kr.iloc[i, 3] in ['은행']:
        wics_kr.iloc[i, 2] = '은행'
    elif wics_kr.iloc[i, 3] in ['증권']:
        wics_kr.iloc[i, 2] = '증권'
    elif wics_kr.iloc[i, 3] in ['창업투자', '카드', '기타금융']:
        wics_kr.iloc[i, 2] = '다각화된금융'
    elif wics_kr.iloc[i, 3] in ['손해보험', '생명보험']:
        wics_kr.iloc[i, 2] = '보험'
    elif wics_kr.iloc[i, 3] in ['부동산']:
        wics_kr.iloc[i, 2] = '부동산'
    elif wics_kr.iloc[i, 3] in ['IT서비스', '소프트웨어']:
        wics_kr.iloc[i, 2] = '소프트웨어와서비스'
    elif wics_kr.iloc[i, 3] in ['통신장비', '핸드셋', '컴퓨터와주변기기', '전자장비와기기', '사무용전자제품']:
        wics_kr.iloc[i, 2] = '기술하드웨어와장비'
    elif wics_kr.iloc[i, 3] in ['반도체와반도체장비']:
        wics_kr.iloc[i, 2] = '반도체와반도체장비'
    elif wics_kr.iloc[i, 3] in ['전자제품', '전기제품']:
        wics_kr.iloc[i, 2] = '전자와전기제품'
    elif wics_kr.iloc[i, 3] in ['디스플레이패널', '디스플레이장비및부품']:
        wics_kr.iloc[i, 2] = '디스플레이'
    elif wics_kr.iloc[i, 3] in ['다각화된통신서비스', '무선통신서비스']:
        wics_kr.iloc[i, 2] = '전기통신서비스'
    elif wics_kr.iloc[i, 3] in ['광고', '방송과엔터테인먼트', '출판', '게임엔터테인먼트', '양방향미디어와서비스']:
        wics_kr.iloc[i, 2] = '미디어와엔터테인먼트'
    elif wics_kr.iloc[i, 3] in ['전기유틸리티', '가스유틸리티', '복합유틸리티', '수도유틸리티', '독립전력생산및에너지거래']:
        wics_kr.iloc[i, 2] = '유틸리티'
    elif wics_kr.iloc[i, 3] in ['미분류']:
        wics_kr.iloc[i, 2] = '미분류'
    else:
        pass

In [ ]:
#대분류
for i in range(len(wics_kr)):
    if wics_kr.iloc[i, 2] in ['에너지']:
        wics_kr.iloc[i, 1] = '에너지'
    elif wics_kr.iloc[i, 2] in ['소재']:
        wics_kr.iloc[i, 1] = '소재'
    elif wics_kr.iloc[i, 2] in ['자본재', '상업서비스와공급품', '운송']:
        wics_kr.iloc[i, 1] = '산업재'
    elif wics_kr.iloc[i, 2] in ['자동차와부품', '내구소비재와의류', '호텔,레스토랑,레저등', '소매(유통)', '교육서비스']:
        wics_kr.iloc[i, 1] = '경기관련소비재'
    elif wics_kr.iloc[i, 2] in ['식품과기본식료품소매', '식품,음료,담배', '가정용품과개인용품']:
        wics_kr.iloc[i, 1] = '필수소비재'
    elif wics_kr.iloc[i, 2] in ['건강관리장비와서비스', '제약과생물공학']:
        wics_kr.iloc[i, 1] = '건강관리'
    elif wics_kr.iloc[i, 2] in ['은행', '증권', '다각화된금융', '보험', '부동산']:
        wics_kr.iloc[i, 1] = '금융'
    elif wics_kr.iloc[i, 2] in ['소프트웨어와서비스', '기술하드웨어와장비', '반도체와반도체장비', '전자와전기제품', '디스플레이']:
        wics_kr.iloc[i, 1] = 'IT'
    elif wics_kr.iloc[i, 2] in ['전기통신서비스', '미디어와엔터테인먼트']:
        wics_kr.iloc[i, 1] = '커뮤니케이션서비스'
    elif wics_kr.iloc[i, 2] in ['유틸리티']:
        wics_kr.iloc[i, 1] = '유틸리티'
    elif wics_kr.iloc[i, 2] in ['미분류']:
        wics_kr.iloc[i, 1] = '미분류'
    else:
        pass

In [ ]:
wics_kr = wics_kr.drop(['iem_eng_nm', 'iem_krl_nm'], axis=1)
wics_kr.to_csv('wics_kr.csv', index=False)

-------

# 2. 해외

## 2-1. Marketscreener

### 라이브러리 가져오기

In [ ]:
import pandas as pd
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings(action='ignore')

### 데이터 가져오기

In [ ]:
#해외주식 기업코드 추출
oss_iem_cd = iem_info['iem_cd'][~iem_info['iem_cd'].isin(wics_kr['iem_cd'])].reset_index(drop=True)

### 크롤링

In [ ]:
def get_oss_sector(company_code):
    driver = wd.Chrome(executable_path='chromedriver.exe')
    url = 'https://www.marketscreener.com/search/?lien=recherche&mots=' + company_code + '&RewriteLast=&noredirect=0&type_recherche=0'
    driver.get(url)
    
    #첫번째 기업 클릭
    driver.find_element_by_xpath('//*[@id="ALNI0"]/tbody/tr[2]/td[3]/a').click()
   
    page_src = driver.page_source
    driver.close()
    soup = BeautifulSoup(page_src, 'html.parser')

    sector = soup.select('#zbCenter > div > span > table:nth-child(7) > tbody > tr:nth-child(2) > td:nth-child(1) > table:nth-child(4) > tbody > tr:nth-child(2) > td > div:nth-child(2) > table > tbody > tr > td:nth-child(1) > a')[0].text
   
    return sector

In [ ]:
#빈 DataFrame 만들기
oss_sector = pd.DataFrame(None, columns=['iem_cd', 'sector'], index=range(len(oss_iem_cd)))

#sector 입력
for i in range(len(oss_iem_cd)):
    oss_sector.iloc[i, 0] = oss_iem_cd[i]
    try:
        oss_sector.iloc[i, 1] = get_oss_sector(oss_iem_cd[i])
    except:
        oss_sector.iloc[i, 1] = None
        
# oss_sector.to_csv('../preprocessed_data/oss_marketscreener.csv', index=False)

## 2-2. Yahoo Finance
- 해외서버 불안정으로 세 번 크롤링함

In [ ]:
#Martketscreener로 검색 안 된 주식들 따로 보기
oss_na_checklist = oss_sector[oss_sector['sector'].isna()]

#NA와 notNA로 나누기
oss_na_checklist_notNA = oss_na_checklist[~oss_na_checklist['sector'].isna()].reset_index(drop=True)
oss_na_checklist_NA = oss_na_checklist[oss_na_checklist['sector'].isna()].reset_index(drop=True)
oss_iem_cd = oss_na_checklist_NA['iem_cd']

### 크롤링

In [ ]:
def find_plz(company_code):
    driver.find_element_by_css_selector('#yfin-usr-qry').clear()
    driver.refresh()
    time.sleep(1)
    driver.find_element_by_css_selector('#yfin-usr-qry').send_keys(company_code)
    time.sleep(3)
    
    try:
        driver.find_element_by_xpath('//*[@id="header-search-form"]/div[2]/div[1]/div/ul[1]/li/div[1]/div[1]').click()    
        time.sleep(3)
        page = driver.page_source
        soup = BeautifulSoup(page)
        isProfile5 = soup.select('#quote-nav > ul > li:nth-child(5) > a > span')[0].text
        isProfile6 = soup.select('#quote-nav > ul > li:nth-child(6) > a > span')[0].text
        isProfile7 = soup.select('#quote-nav > ul > li:nth-child(7) > a > span')[0].text
        
        if isProfile7 == 'Profile': #Company Outlook 잠금 ver.
            driver.find_element_by_xpath('//*[@id="quote-nav"]/ul/li[7]/a/span').click()
            time.sleep(3)
            page = driver.page_source
            soup = BeautifulSoup(page)
            sector = soup.find_all('span', class_='Fw(600)')[0].text
            print(sector)
            return sector
            
        elif isProfile6 == 'Profile':
            driver.find_element_by_xpath('//*[@id="quote-nav"]/ul/li[6]').click()
            time.sleep(3)
            page = driver.page_source
            soup = BeautifulSoup(page)
            sector = soup.find_all('span', class_='Fw(600)')[0].text
            print(sector)
            return sector
        elif isProfile5 == 'Profile':
            driver.find_element_by_xpath('//*[@id="quote-nav"]/ul/li[5]').click()
            time.sleep(3)
            page = driver.page_source
            soup = BeautifulSoup(page)
            sector = soup.find_all('span', class_='Fl(end)')[0].text #category를 가져옴...sector 아님 주의
            isETF = soup.find_all('span', class_='Fl(end)')[5].text #ETF 여부까지!
            sector = sector + ' (' + isETF  + ')'
            print(sector)
            return sector
        else:
            print('check later')
            return 'check later'
        
    except:
        print('None')
        return None

In [ ]:
driver = wd.Chrome(executable_path='chromedriver.exe')
driver.get('https://finance.yahoo.com/')

In [ ]:
empty_lst = []
for i in range(len(oss_iem_cd)):
    empty_lst.append(find_plz(oss_iem_cd[i]))

In [ ]:
#다시 합쳐서 내보내기
oss_na_checklist_NA['sector'] = empty_lst
oss_na_checklist = pd.concat([oss_na_checklist_notNA, oss_na_checklist_NA], axis=0)

# oss_na_checklist.to_csv('../check/oss_yahoo_first.csv', index=False, encoding='cp949')
# oss_na_checklist.to_csv('../check/oss_yahoo_second.csv', index=False, encoding='cp949')
# oss_na_checklist.to_csv('../check/oss_yahoo_third.csv', index=False, encoding='cp949')

### 크롤링 통합

In [ ]:
oss = iem_info[~iem_info['iem_cd'].isin(wics_kr['iem_cd'])]

#국내 J or Q (1. 국내에서 못 다룬 국내주식자료들)
oss_J = oss[oss['iem_cd'].str.startswith('J')]
oss_J = oss_J[~oss_J['iem_cd'].str.startswith('JP')] #JP로 시작하는 것은 제외
oss_J = pd.merge(oss_J, iem_info).sort_values(by='iem_krl_nm')
oss_Q = oss[oss['iem_cd'].str.startswith('Q')]
oss_Q = pd.merge(oss_Q, iem_info).sort_values(by='iem_krl_nm')

oss_J['sector'] = 'ELW'
oss_Q['sector'] = 'ETF' #ETF, ETF는 모두 ETF로 통합

oss_J.reset_index(drop=True, inplace=True)
oss_Q.reset_index(drop=True, inplace=True)
# oss_J.to_csv('../check/oss_J.csv', encoding='cp949', index=False)
# oss_Q.to_csv('../check/oss_Q.csv', encoding='cp949', index=False)

print(oss_J.shape) # J는 주식워런트증권
print(oss_Q.shape) # Q는 ETF / ETN / DLS 같은 것들

In [ ]:
#해외1. MarketScreener
#병합 알고리즘 흐름
# - oss_marketscreener -> oss_marketscreener_notNA + oss_marketscreener_NA
# - oss_marketscreener_NA -> (yahoo와 merge 후) tmp_notNA + tmp_NA
# - oss_marketscreener_notNA와 tmp_notNA 병합 / oss_marketscreener_NA -> tmp_NA

oss_marketscreener = pd.read_csv('../preprocessed_data/oss_marketscreener.csv', encoding='cp949')

In [ ]:
#해외2-1. Yahoo Finance (해외1에서 미분류된  애들 분류하기)
#첫번째 yahoo 크롤링 / NA 223개 줄어듬
oss_marketscreener_notNA = oss_marketscreener[~oss_marketscreener['sector'].isna()]
oss_marketscreener_NA = oss_marketscreener[oss_marketscreener['sector'].isna()]
print(oss_marketscreener_notNA.shape)
print(oss_marketscreener_NA.shape)

oss_yahoo = pd.read_csv('../preprocessed_data/oss_yahoo_first.csv', encoding='cp949') #야후에서 처음으로 크롤링한 것
tmp = pd.merge(oss_marketscreener_NA[['iem_cd']], oss_yahoo, how='left')
tmp_notNA = tmp[~tmp['sector'].isna()]
tmp_NA = tmp[tmp['sector'].isna()]
oss_marketscreener_notNA = pd.concat([oss_marketscreener_notNA, tmp_notNA], axis=0)
oss_marketscreener_NA = tmp_NA
print(oss_marketscreener_notNA.shape)
print(oss_marketscreener_NA.shape)

oss_marketscreener_NA.reset_index(drop=True).to_csv('../check/oss_marketscreener_NA.csv', encoding='cp949', index=False)

In [ ]:
#해외2-2. Yahoo Finance (해외1-1에서 미분류된  애들 분류하기)
#두번째 / NA 126개 줄어듬
oss_marketscreener_NA = pd.read_csv('../check/oss_marketscreener_NA.csv', encoding='cp949')
oss_yahoo_again = pd.read_csv('../check/oss_yahoo_second.csv', encoding='cp949') #두번째 yahoo 크롤링

tmp = pd.merge(oss_marketscreener_NA[['iem_cd']], oss_yahoo_again, how='left')
tmp_notNA = tmp[~tmp['sector'].isna()]
tmp_NA = tmp[tmp['sector'].isna()]
oss_marketscreener_notNA = pd.concat([oss_marketscreener_notNA, tmp_notNA], axis=0)
oss_marketscreener_NA = tmp_NA
print(oss_marketscreener_notNA.shape)
print(oss_marketscreener_NA.shape)

oss_marketscreener_NA.reset_index(drop=True).to_csv('../check/oss_marketscreener_NA.csv', encoding='cp949', index=False)

In [ ]:
#해외2-3. Yahoo Finance (해외1-1에서 미분류된  애들 분류하기)
#세번째 / NA 7개 줄어듬
oss_marketscreener_NA = pd.read_csv('../check/oss_marketscreener_NA.csv', encoding='cp949')
oss_yahoo_NA = pd.read_csv('../check/oss_yahoo_third.csv', encoding='cp949')  #세번째 yahoo 크롤링

tmp = pd.merge(oss_marketscreener_NA[['iem_cd']], oss_yahoo_NA, how='left')
tmp_notNA = tmp[~tmp['sector'].isna()]
tmp_NA = tmp[tmp['sector'].isna()]
oss_marketscreener_notNA = pd.concat([oss_marketscreener_notNA, tmp_notNA], axis=0)
oss_marketscreener_NA = tmp_NA
print(oss_marketscreener_notNA.shape)
print(oss_marketscreener_NA.shape)

In [ ]:
#해외 ETF 구별하기
oss_na = pd.merge(oss_marketscreener_NA[['iem_cd', 'sector', 'title']], iem_info)

In [ ]:
def get_ETF(data, word):
    oss_na_ETF = data[data['iem_krl_nm'].str.contains(word)].reset_index(drop=True)
    oss_na_notETF = data[~data['iem_krl_nm'].str.contains(word)].reset_index(drop=True)
    oss_na_ETF['sector'] = 'ETF'
    oss_na = pd.concat([oss_na_ETF, oss_na_notETF])
    oss_na.reset_index(drop=True, inplace=True)
    return oss_na

oss_na = get_ETF(oss_na, 'ETF')
oss_na = get_ETF(oss_na, 'ETN')
oss_na = get_ETF(oss_na, ' 3X')
oss_na = get_ETF(oss_na, ' 2X')

In [ ]:
oss_marketscreener_notNA = pd.concat([oss_marketscreener_notNA, oss_na[~oss_na['sector'].isna()]], axis=0).reset_index(drop=True)
oss_marketscreener_NA = oss_na[oss_na['sector'].isna()]
print(oss_marketscreener_notNA.shape)
print(oss_marketscreener_NA.shape)

#내보내기
# oss_marketscreener_notNA.reset_index(drop=True).to_csv('../check/oss_marketscreener_notNA.csv', encoding='cp949', index=False)
# oss_marketscreener_NA.reset_index(drop=True).to_csv('../check/oss_marketscreener_NA.csv', encoding='cp949', index=False)

In [ ]:
#통합
oss_marketscreener = pd.concat([oss_marketscreener_notNA, oss_marketscreener_NA], axis=0).reset_index(drop=True)
oss_merged = pd.merge(oss_marketscreener[['iem_cd','sector']], iem_info, how='left')
oss_merged = pd.concat([oss_merged, oss_J, oss_Q]).reset_index(drop=True) #oss_J, oss_Q도 합치기
oss_merged.to_csv('../preprocessed_data/oss_merged.csv', index=False, encoding='cp949')

### 분류하기

In [15]:
#데이터 가져오기
oss_merged = pd.read_csv('../preprocessed_data/oss_merged.csv', encoding='cp949')
oss_merged_notNA = oss_merged[~oss_merged['sector'].isna()]

oss_ms_cat = pd.read_csv('../check/oss_marketscreener_cat_together.csv', encoding='cp949') #카테고리 분류(eng->kr)
oss_ms_cat = oss_ms_cat[['sector', 'wics']] #ms = marketscreener

oss_ms_cat_self = pd.read_csv('../check/oss_marketscreener_NA_직접입력.csv', encoding='cp949') #검색이 안되는 회사들(36개) 직접 서칭 후 분류
oss_ms_cat_self = oss_ms_cat_self[['iem_cd', 'wics', 'iem_eng_nm', 'iem_krl_nm']]

In [16]:
#소분류 기준 만들기
oss_ms_cat_dict = {}
for i in range(len(oss_ms_cat)):
    oss_ms_cat_dict[oss_ms_cat.iloc[i,0]] = oss_ms_cat.iloc[i,1]
oss_ms_cat_dict['ETF'] = 'etf'
oss_ms_cat_dict['ELW'] = 'ELW'

In [17]:
def get_ms_wics(oss_merged_notNA):
    try:
        return oss_ms_cat_dict[oss_merged_notNA['sector']]
    except:
        return None

In [18]:
#소분류
oss_merged_notNA['wics'] = oss_merged_notNA.apply(get_ms_wics, axis=1) #ms_cat으로만 해도 충분함 (yh_cat이랑 겹침)
oss_merged_notNA.loc[oss_merged_notNA['wics'].isna(), 'wics'] = 'etf' #나머지 NA는 모두 ETF임을 확인함.
oss_merged_notNA = oss_merged_notNA[['iem_cd', 'wics', 'iem_eng_nm', 'iem_krl_nm']]

oss_wics = pd.concat([oss_ms_cat_self, oss_merged_notNA], axis=0)
oss_wics.reset_index(drop=True, inplace=True)
oss_wics.columns = ['iem_cd', 'cat_3', 'iem_eng_nm', 'iem_krl_nm']

#소분류 수정
oss_wics.iloc[[1043, 1112, 1196, 1197, 1231, 1239, 1263, 1041, 1045, 1076, 1108, 1149, 1172], 
              1] = ['해운사', '해운사', '해운사', '해운사', '상업서비스와공급품', '상업서비스와공급품', '기계', '전문소매',
                    '인터넷과카탈로그소매', '섬유,의류,신발,호화품', '자동차', '인터넷과카탈로그소매', '호텔,레스토랑,레저']

<ipython-input-18-5d4247b0e149>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oss_merged_notNA['wics'] = oss_merged_notNA.apply(get_ms_wics, axis=1) #ms_cat으로만 해도 충분함 (yh_cat이랑 겹침)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-18-5d4247b0e149>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [19]:
#중분류 & 대분류
oss_wics['cat_1'] = None
oss_wics['cat_2'] = None
oss_wics = oss_wics[['iem_cd', 'iem_eng_nm', 'iem_krl_nm', 'cat_1', 'cat_2', 'cat_3']]

#중분류
for i in range(len(oss_wics)):
    if oss_wics.iloc[i, 5] in ['에너지장비및서비스', '석유와가스']:
        oss_wics.iloc[i, 4] = '에너지'
    elif oss_wics.iloc[i, 5] in ['화학', '포장재', '비철금속', '철강', '종이와목재']:
        oss_wics.iloc[i, 4] = '소재'
    elif oss_wics.iloc[i, 5] in ['우주항공과국방', '건축제품', '건축자재', '건설', '가구', '전기장비', '복합기업', '기계', '조선', '무역회사와판매업체']:
        oss_wics.iloc[i, 4] = '자본재'
    elif oss_wics.iloc[i, 5] in ['상업서비스와공급품']:
        oss_wics.iloc[i, 4] = '상업서비스와공급품'
    elif oss_wics.iloc[i, 5] in ['항공화물운송과물류', '항공사', '해운사', '도로와철도운송', '운송인프라']:
        oss_wics.iloc[i, 4] = '운송'
    elif oss_wics.iloc[i, 5] in ['자동차부품', '자동차']:
        oss_wics.iloc[i, 4] = '자동차와부품'
    elif oss_wics.iloc[i, 5] in ['가정용기기와용품', '레저용장비와제품', '섬유,의류,신발,호화품', '화장품', '문구류']:
        oss_wics.iloc[i, 4] = '내구소비재와의류'
    elif oss_wics.iloc[i, 5] in ['호텔,레스토랑,레저', '다각화된소비자서비스']:
        oss_wics.iloc[i, 4] = '호텔,레스토랑,레저등'
    elif oss_wics.iloc[i, 5] in ['판매업체', '인터넷과카탈로그소매', '백화점과일반상점', '전문소매']:
        oss_wics.iloc[i, 4] = '소매(유통)'
    elif oss_wics.iloc[i, 5] in ['교육서비스']:
        oss_wics.iloc[i, 4] = '교육서비스'
    elif oss_wics.iloc[i, 5] in ['식품과기본식료품소매']:
        oss_wics.iloc[i, 4] = '식품과기본식료품소매'
    elif oss_wics.iloc[i, 5] in ['음료', '식품', '담배']:
        oss_wics.iloc[i, 4] = '식품,음료,담배'
    elif oss_wics.iloc[i, 5] in ['가정용품']:
        oss_wics.iloc[i, 4] = '가정용품과개인용품'
    elif oss_wics.iloc[i, 5] in ['건강관리장비와용품', '건강관리업체및서비스', '건강관리기술']:
        oss_wics.iloc[i, 4] = '건강관리장비와서비스'
    elif oss_wics.iloc[i, 5] in ['생물공학', '제약', '생명과학도구및서비스']:
        oss_wics.iloc[i, 4] = '제약과생물공학'
    elif oss_wics.iloc[i, 5] in ['은행']:
        oss_wics.iloc[i, 4] = '은행'
    elif oss_wics.iloc[i, 5] in ['증권']:
        oss_wics.iloc[i, 4] = '증권'
    elif oss_wics.iloc[i, 5] in ['창업투자', '카드', '기타금융']:
        oss_wics.iloc[i, 4] = '다각화된금융'
    elif oss_wics.iloc[i, 5] in ['손해보험', '생명보험']:
        oss_wics.iloc[i, 4] = '보험'
    elif oss_wics.iloc[i, 5] in ['부동산']:
        oss_wics.iloc[i, 4] = '부동산'
    elif oss_wics.iloc[i, 5] in ['IT서비스', '소프트웨어']:
        oss_wics.iloc[i, 4] = '소프트웨어와서비스'
    elif oss_wics.iloc[i, 5] in ['통신장비', '핸드셋', '컴퓨터와주변기기', '전자장비와기기', '사무용전자제품']:
        oss_wics.iloc[i, 4] = '기술하드웨어와장비'
    elif oss_wics.iloc[i, 5] in ['반도체와반도체장비']:
        oss_wics.iloc[i, 4] = '반도체와반도체장비'
    elif oss_wics.iloc[i, 5] in ['전자제품', '전기제품']:
        oss_wics.iloc[i, 4] = '전자와전기제품'
    elif oss_wics.iloc[i, 5] in ['디스플레이패널', '디스플레이장비및부품']:
        oss_wics.iloc[i, 4] = '디스플레이'
    elif oss_wics.iloc[i, 5] in ['다각화된통신서비스', '무선통신서비스']:
        oss_wics.iloc[i, 4] = '전기통신서비스'
    elif oss_wics.iloc[i, 5] in ['광고', '방송과엔터테인먼트', '출판', '게임엔터테인먼트', '양방향미디어와서비스']:
        oss_wics.iloc[i, 4] = '미디어와엔터테인먼트'
    elif oss_wics.iloc[i, 5] in ['전기유틸리티', '가스유틸리티', '복합유틸리티', '수도유틸리티', '독립전력생산및에너지거래']:
        oss_wics.iloc[i, 4] = '유틸리티'
    elif oss_wics.iloc[i, 5] in ['미분류']:
        oss_wics.iloc[i, 4] = '미분류'
    elif oss_wics.iloc[i, 5] in ['ELW']:
        oss_wics.iloc[i, 4] = 'ELW'        
    elif oss_wics.iloc[i, 5] in ['etf']:
        oss_wics.iloc[i, 4] = 'ETF'
    else:
        oss_wics.iloc[i, 4] = None   

In [20]:
#대분류
for i in range(len(oss_wics)):
    if oss_wics.iloc[i, 4] in ['에너지']:
        oss_wics.iloc[i, 3] = '에너지'
    elif oss_wics.iloc[i, 4] in ['소재']:
        oss_wics.iloc[i, 3] = '소재'
    elif oss_wics.iloc[i, 4] in ['자본재', '상업서비스와공급품', '운송']:
        oss_wics.iloc[i, 3] = '산업재'
    elif oss_wics.iloc[i, 4] in ['자동차와부품', '내구소비재와의류', '호텔,레스토랑,레저등', '소매(유통)', '교육서비스']:
        oss_wics.iloc[i, 3] = '경기관련소비재'
    elif oss_wics.iloc[i, 4] in ['식품과기본식료품소매', '식품,음료,담배', '가정용품과개인용품']:
        oss_wics.iloc[i, 3] = '필수소비재'
    elif oss_wics.iloc[i, 4] in ['건강관리장비와서비스', '제약과생물공학']:
        oss_wics.iloc[i, 3] = '건강관리'
    elif oss_wics.iloc[i, 4] in ['은행', '증권', '다각화된금융', '보험', '부동산']:
        oss_wics.iloc[i, 3] = '금융'
    elif oss_wics.iloc[i, 4] in ['소프트웨어와서비스', '기술하드웨어와장비', '반도체와반도체장비', '전자와전기제품', '디스플레이']:
        oss_wics.iloc[i, 3] = 'IT'
    elif oss_wics.iloc[i, 4] in ['전기통신서비스', '미디어와엔터테인먼트']:
        oss_wics.iloc[i, 3] = '커뮤니케이션서비스'
    elif oss_wics.iloc[i, 4] in ['유틸리티']:
        oss_wics.iloc[i, 3] = '유틸리티'
    elif oss_wics.iloc[i, 4] in ['미분류']:
        oss_wics.iloc[i, 3] = '미분류'
    elif oss_wics.iloc[i, 4] in ['ELW']:
        oss_wics.iloc[i, 3] = 'ELW'        
    elif oss_wics.iloc[i, 4] in ['ETF']:
        oss_wics.iloc[i, 3] = 'ETF'                
    else:
        oss_wics.iloc[i, 3] = None

In [21]:
#ETF 중분류 & 대분류
ETForNot = oss_wics['cat_1']=='ETF' 
INV_eng = oss_wics['iem_eng_nm'].str.contains('Inverse|INVERSE| INV |INVRS')
INV_kr = oss_wics['iem_krl_nm'].str.contains('인버스')

times2_eng = oss_wics['iem_eng_nm'].str.contains('2x|2X|leverage|Leverage|LEVERAGE')
times2_kr = oss_wics['iem_krl_nm'].str.contains('2x|2X|레버리지')
times3_eng = oss_wics['iem_eng_nm'].str.contains('3x|3X')
times3_kr = oss_wics['iem_krl_nm'].str.contains('3x|3X')

oss_wics.loc[oss_wics['cat_1'] == 'ETF', 'cat_2'] = 'ETF(일반)'
oss_wics.loc[ETForNot & (INV_eng|INV_kr), 'cat_2'] = 'ETF(인버스)'

oss_wics.loc[oss_wics['cat_1'] == 'ETF', 'cat_3'] = 'ETF(1배)'
oss_wics.loc[ETForNot & (times2_eng|times2_kr), 'cat_3'] = 'ETF(2배)'
oss_wics.loc[ETForNot & (times3_eng|times3_kr), 'cat_3'] = 'ETF(3배)'

In [22]:
oss_wics.drop(['iem_eng_nm', 'iem_krl_nm'], axis=1, inplace=True)
oss_wics.to_csv('../preprocessed_data/wics_oss.csv', index=False, encoding='cp949')


------

# WICS 통합본

In [24]:
wics_oss = pd.read_csv('../preprocessed_data/wics_oss.csv', encoding='cp949')
wics_kr = pd.read_csv('../preprocessed_data/wics_kr.csv')

In [25]:
wics = pd.concat([wics_kr, wics_oss])

In [ ]:
wics = pd.merge(wics, iem_info, on='iem_cd')

In [ ]:
for i in range(len(wics)):
    if ('SHORT' in wics.iloc[i, 4]) | ('Sht' in wics.iloc[i, 4]) | ('숏' in wics.iloc[i, 5]):
        wics.iloc[i, 2] = 'ETF(인버스)'

for i in range(len(wics)):
    if ('DOUBLE' in wics.iloc[i, 4]) | ('울트라' in wics.iloc[i, 5]):
        wics.iloc[i, 3] = 'ETF(2배)'
    elif ('UltP' in wics.iloc[i, 4]) | ('울트라프로' in wics.iloc[i, 5]):
        wics.iloc[i, 3] = 'ETF(3배)'

In [ ]:
wics = wics.drop(['iem_krl_nm', 'iem_eng_nm'], axis=1)

In [27]:
wics.to_csv('../preprocessed_data/wics.csv', index=False, encoding='cp949')

--------

# 3. 기타 그외

### 3-1. KOSPI

In [ ]:
kospi = pd.DataFrame(columns=['날짜', '체결가', '전일비', '등락률', '거래량(천주)', '거래대금(백만)'])

In [ ]:
for i in range(1, 82):
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=' + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    kospi = pd.concat([kospi, pd.read_html(str(soup))[0].dropna()])

In [ ]:
kospi = kospi.reset_index(drop=True)

In [ ]:
kospi = kospi[:481]

In [ ]:
kospi['날짜'] = pd.to_datetime(kospi['날짜'])

In [ ]:
kospi = kospi.sort_values('날짜').reset_index(drop=True)

In [ ]:
kospi = kospi[['날짜', '체결가']]

In [ ]:
kospi.columns = ['orr_dt', 'orr_pr']

In [ ]:
kospi.to_csv('kospi.csv', index=False)